# 测试集总体准确率评估指标

分析测试集预测结果表格，计算总体准确率评估指标和各类别准确率评估指标。

同济子豪兄：https://space.bilibili.com/1900783

## 导入工具包

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## 载入类别名称和ID

In [2]:
idx_to_labels = np.load(r'E:\MV-Code-202018010103-Lucy\main\Train_Custom_Dataset\图像分类\3-【Pytorch】迁移学习训练自己的图像分类模型\idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

['parasitized', 'uninfected']


## 载入测试集预测结果表格

In [3]:
df = pd.read_csv('测试集预测结果.csv')

In [4]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-n预测正确,parasitized-预测置信度,uninfected-预测置信度
0,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.965240,0.034760
1,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,1,uninfected,0,parasitized,True,0.383567,0.616433
2,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.780675,0.219325
3,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.994083,0.005918
4,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.997244,0.002756
...,...,...,...,...,...,...,...,...,...,...
5507,D:\dataset\sr\val\uninfected\C99P60ThinF_IMG_2...,1,uninfected,1,uninfected,0,parasitized,True,0.078119,0.921881
5508,D:\dataset\sr\val\uninfected\C99P60ThinF_IMG_2...,1,uninfected,0,parasitized,1,uninfected,True,0.636197,0.363803
5509,D:\dataset\sr\val\uninfected\C99P60ThinF_IMG_2...,1,uninfected,1,uninfected,0,parasitized,True,0.453985,0.546015
5510,D:\dataset\sr\val\uninfected\C99P60ThinF_IMG_2...,1,uninfected,1,uninfected,0,parasitized,True,0.174118,0.825882


## 准确率

In [5]:
sum(df['标注类别名称'] == df['top-1-预测名称']) / len(df)

0.9023947750362845

## top-n准确率

In [6]:
sum(df['top-n预测正确']) / len(df)

1.0

## 各类别其它评估指标

公众号 人工智能小技巧 回复 混淆矩阵

看子豪兄视频和笔记讲解

In [7]:
from sklearn.metrics import classification_report

In [8]:
print(classification_report(df['标注类别名称'], df['top-1-预测名称'], target_names=classes))

              precision    recall  f1-score   support

 parasitized       0.92      0.88      0.90      2756
  uninfected       0.89      0.92      0.90      2756

    accuracy                           0.90      5512
   macro avg       0.90      0.90      0.90      5512
weighted avg       0.90      0.90      0.90      5512


macro avg 宏平均：直接将每一类的评估指标求和取平均（算数平均值）

weighted avg 加权平均：按样本数量（support）加权计算评估指标的平均值

In [9]:
report = classification_report(df['标注类别名称'], df['top-1-预测名称'], target_names=classes, output_dict=True)
del report['accuracy']
df_report = pd.DataFrame(report).transpose()

In [10]:
df_report

,precision,recall,f1-score,support
parasitized,0.919440,0.882075,0.900370,2756.0
uninfected,0.886681,0.922714,0.904339,2756.0
macro avg,0.903060,0.902395,0.902354,5512.0
weighted avg,0.903060,0.902395,0.902354,5512.0


## 补充：各类别准确率（其实就是recall）

In [11]:
accuracy_list = []
for fruit in tqdm(classes):
    df_temp = df[df['标注类别名称']==fruit]
    accuracy = sum(df_temp['标注类别名称'] == df_temp['top-1-预测名称']) / len(df_temp)
    accuracy_list.append(accuracy)

100%|██████████| 2/2 [00:00<00:00, 357.74it/s]


In [12]:
# 计算 宏平均准确率 和 加权平均准确率
acc_macro = np.mean(accuracy_list)
acc_weighted = sum(accuracy_list * df_report.iloc[:-2]['support'] / len(df))

accuracy_list.append(acc_macro)
accuracy_list.append(acc_weighted)

df_report['accuracy'] = accuracy_list

In [13]:
df_report

,precision,recall,f1-score,support,accuracy
parasitized,0.919440,0.882075,0.900370,2756.0,0.882075
uninfected,0.886681,0.922714,0.904339,2756.0,0.922714
macro avg,0.903060,0.902395,0.902354,5512.0,0.902395
weighted avg,0.903060,0.902395,0.902354,5512.0,0.902395


In [14]:
df_report.to_csv('各类别准确率评估指标.csv', index_label='类别')